# The Benefit of Data Sharing
Godwin et al. (2025) reviewed the open-science practices in recent **visual search** literature.<br>
They gracefully provided the dataset of articles they reviewed ([link](https://osf.io/5tmey/overview)), allowing others to explore the data further.<br>
Here, we analyze their dataset to examine whether sharing data is associated with increased citations. We rely on Godwin et al.'s classification of articles into four categories based on their data-sharing practices:
1. No data shared
2. Per-subject data
3. Per-trial data
4. Per-fixation data

We use the [OpenAlex](https://openalex.org/) API to retrieve citation counts for each of their articles, and compare the [FWCI](https://help.openalex.org/hc/en-us/articles/24735753007895-Field-Weighted-Citation-Impact-FWCI) and [MNCS](https://open.leidenranking.com/information/indicators) between these sharing categories.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import multipletests
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from load_data import *
from fetch_metadata import fetch_all_metadata

In [2]:
FONT_FAMILY = "sans-serif"
TITLE_FONT = dict(family=FONT_FAMILY, size=20, color="black")
AXIS_TITLE_FONT = dict(family=FONT_FAMILY, size=16, color="black")
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=14, color="black")
LEGEND_FONT = dict(family=FONT_FAMILY, size=14, color="black")

## Prepare Data
### Load Godwin et al. (2025) Dataset

In [3]:
godwin = load_godwin2025()
godwin_subset = (
    godwin
    # exclusion criteria from Godwin et al. 2025:
    .loc[(godwin["YEAR_PUBLISHED"] >= 2017) & (godwin["YEAR_PUBLISHED"] <= 2022)]
    .loc[godwin["IS_PRIMARY_RESEARCH_HUMAN"] == "YES"]
    .loc[godwin["IS_VISUAL_SEARCH"] == "YES"]
    .loc[godwin["IS_EYE_TRACKING"] == "YES"]
    .drop(columns=[
        col for col in godwin.columns if col not in [
            "PAPER_LINK",
            "CLAIMED_TO_SHARE", "SHARING_LINK", "ACTUALLY_SHARED",
            "EXPERIMENT", "MATERIALS", "CODE", "CODEBOOK_GUIDE",
            "BY_FIXATION", "BY_TRIAL", "BY_PPT",
            "data_sharing_class"
        ]
    ])
)
print(f"Full Godwin et al. dataset: {len(godwin)} articles")
print(f"Subset used for analysis: {len(godwin_subset)} articles")

Full Godwin et al. dataset: 513 articles
Subset used for analysis: 251 articles


### Fetch Metadata & Citation Counts from OpenAlex

In [4]:
metadata = fetch_all_metadata(godwin_subset["PAPER_LINK"].tolist(), sleep_period=0.01, verbose=True)

  0%|          | 0/251 [00:00<?, ?it/s]C:\Users\nirjo\Documents\University\PhD\Projects\eye_movement_data_sharing\fetch_metadata.py:27: RuntimeWarning: coroutine 'sleep' was never awaited
  sleep(sleep_period)     # to respect rate limits of 100 requests per second
  1%|          | 2/251 [00:02<05:35,  1.35s/it]

Error fetching metadata for link https://jov.arvojournals.org/article.aspx?articleid=2770293: Failed to fetch URL content: https://jov.arvojournals.org/article.aspx?articleid=2770293


 27%|██▋       | 68/251 [01:13<02:47,  1.10it/s]

Error fetching metadata for link 10.1177/1747021820945604: 404 Client Error: Not Found for url: https://api.openalex.org/works/doi%3A10.1177%2F1747021820945604


 32%|███▏      | 80/251 [01:29<04:27,  1.56s/it]

...

 41%|████      | 103/251 [01:56<02:28,  1.01s/it]

Error fetching metadata for link https://www.sciencedirect.com/science/article/abs/pii/S0010027719303233?via%3Dihub: Failed to fetch URL content: https://www.sciencedirect.com/science/article/abs/pii/S0010027719303233?via%3Dihub


 45%|████▍     | 112/251 [02:01<01:33,  1.49it/s]

Error fetching metadata for link 10.16910/jemr.10.1.5: 404 Client Error: Not Found for url: https://api.openalex.org/works/doi%3A10.16910%2Fjemr.10.1.5


100%|██████████| 251/251 [04:08<00:00,  1.01it/s]


In [5]:
combined = (
    metadata
    .dropna(subset=["DOI"])     # drop entries with unsuccessful metadata fetch
    .merge(godwin_subset, on="PAPER_LINK")
    .drop_duplicates(subset="DOI")
    .loc[lambda df: df["IsRetracted"] == False]   # drop retracted articles
    .assign(
        is_sharing_data=lambda df: df["data_sharing_class"] != "NONE",
        is_sharing_anything=lambda df: (df[[
            "EXPERIMENT", "MATERIALS", "CODE", "CODEBOOK_GUIDE", "BY_FIXATION", "BY_TRIAL", "BY_PPT"
        ]] == "YES").any(axis=1)
    )
)

# print summary of fetch results
num_fails = metadata["DOI"].isna().sum()
num_dups = metadata.loc[metadata["DOI"].notna(), :].duplicated(subset="DOI").sum()
num_retracted = metadata["IsRetracted"].sum()
num_sharing_anything = combined["is_sharing_anything"].sum()
num_sharing_data = combined["is_sharing_data"].sum()

print("OpenAlex Metadata Fetch Summary:")
print(f"Total articles queried: {len(godwin_subset)}")
print(f"\t- Failed fetches: {num_fails}")
print(f"\t- Duplicate DOIs: {num_dups}")
print(f"\t- Retracted articles: {num_retracted}")
print(f"Final dataset for analysis: {len(combined)} articles")
print(f"\t- Articles sharing any materials: {num_sharing_anything} ({num_sharing_anything / len(combined):.1%})")
print(f"\t- Articles sharing data: {num_sharing_data} ({num_sharing_data / len(combined):.1%})")

OpenAlex Metadata Fetch Summary:
Total articles queried: 251
	- Failed fetches: 13
	- Duplicate DOIs: 6
	- Retracted articles: 0
Final dataset for analysis: 232 articles
	- Articles sharing any materials: 82 (35.3%)
	- Articles sharing data: 79 (34.1%)


## Analysis

In [6]:
def calculate_h_index(citations: pd.Series) -> int:
    """Calculates h-index from a Series of citation counts."""
    if citations.empty:
        return 0
    # Sort descending (highest citations first)
    sorted_cits = sorted(citations.dropna().astype(int), reverse=True)
    # Count how many papers have citations >= their rank
    return sum(c >= i + 1 for i, c in enumerate(sorted_cits))

In [7]:
metrics_per_sharing_class = (
    combined[[
        "data_sharing_class", "FieldWeightedCitationIndex", "MeanNormalizedCitationScore", "TotalCitations"
    ]]
    .groupby("data_sharing_class")
    .agg({
        "TotalCitations": [calculate_h_index],
        "FieldWeightedCitationIndex": ["count", "median", "mean", "std",],
        "MeanNormalizedCitationScore": ["median", "mean", "std",],
    })
)
metrics_for_any_data_sharing = (
    combined.loc[
        combined["is_sharing_data"],
        ["FieldWeightedCitationIndex", "MeanNormalizedCitationScore", "TotalCitations"]
    ].agg({
        "TotalCitations": [calculate_h_index],
        "FieldWeightedCitationIndex": ["count", "median", "mean", "std",],
        "MeanNormalizedCitationScore": ["median", "mean", "std",],
    })
    .unstack()
    .dropna()
    .rename("ANY")
    .to_frame().T
)

sharing_class_order = {"FIXATION": 0, "TRIAL": 1, "PPT": 2, "ANY": 3, "NONE": 4}
sharing_metrics_summary = (
    pd.concat([metrics_per_sharing_class, metrics_for_any_data_sharing], axis=0)
    .sort_index(key=lambda idx: idx.map(sharing_class_order))
)
sharing_metrics_summary.columns = sharing_metrics_summary.columns.map({
    ("FieldWeightedCitationIndex", "count"): "count",
    ("FieldWeightedCitationIndex", "median"): "fwci_median",
    ("FieldWeightedCitationIndex", "mean"): "fwci_mean",
    ("FieldWeightedCitationIndex", "std"): "fwci_std",
    ("MeanNormalizedCitationScore", "median"): "mncs_median",
    ("MeanNormalizedCitationScore", "mean"): "mncs_mean",
    ("MeanNormalizedCitationScore", "std"): "mncs_std",
}).map(lambda col: "h_index" if pd.isna(col) else col)

sharing_metrics_summary

,h_index,count,fwci_median,fwci_mean,fwci_std,mncs_median,mncs_mean,mncs_std
FIXATION,16.0,31.0,1.631840,2.332376,1.946460,0.933962,1.223868,0.914469
TRIAL,11.0,25.0,1.022195,1.588806,1.501962,0.607187,0.953716,0.945922
PPT,14.0,23.0,1.737731,2.076712,1.494861,0.850062,1.113471,1.012047
ANY,24.0,79.0,1.506314,2.022635,1.697323,0.789343,1.106236,0.948055
NONE,31.0,153.0,1.129736,1.531699,1.348896,0.726415,0.947771,0.836201


### (1) Compare Sharing and Non-Sharing Articles
#### (1A) Field-Weighted Citation Index (FWCI)

In [8]:
fwci_pair_test = stats.mannwhitneyu(
    combined.loc[~combined["is_sharing_data"], "FieldWeightedCitationIndex"],
    combined.loc[combined["is_sharing_data"], "FieldWeightedCitationIndex"],
    alternative="less"
)
fwci_pair_test

MannwhitneyuResult(statistic=np.float64(5126.5), pvalue=np.float64(0.029252484326358907))

#### (1B) Mean Normalized Citation Score (MNCS)

In [9]:
mncs_pair_test = stats.mannwhitneyu(
    combined.loc[~combined["is_sharing_data"], "MeanNormalizedCitationScore"],
    combined.loc[combined["is_sharing_data"], "MeanNormalizedCitationScore"],
    alternative="less"
)
mncs_pair_test

MannwhitneyuResult(statistic=np.float64(5520.5), pvalue=np.float64(0.14038480728155245))

#### Visualizing Share/Non-Share Comparison

In [10]:
plot_data = (
    combined
    .melt(
        id_vars=['is_sharing_data'],
        value_vars=['FieldWeightedCitationIndex', 'MeanNormalizedCitationScore'],
        var_name='name',
        value_name='value'
    )
    .replace({
        'FieldWeightedCitationIndex': 'FWCI',
        'MeanNormalizedCitationScore': 'MNCS',
    })
)

pairwise_fig = go.Figure()
for is_share in plot_data["is_sharing_data"].unique():
    subset = plot_data[plot_data["is_sharing_data"] == is_share]
    subset_name = 'Sharing' if is_share else 'Not Sharing'
    color = 'gray' if is_share else 'lightgray'
    pairwise_fig.add_trace(go.Violin(
        x=subset['name'], y=subset['value'],
        name=subset_name, legendgroup=subset_name, scalegroup=subset_name,
        side='positive' if is_share else 'negative',
        # opacity=1.0 if is_share else 0.75,
        fillcolor=color, line_color=color,
        width=1.0, spanmode="hard",
        points=False, pointpos=0, jitter=0.5,
        box=dict(visible=False, width=0.5, line=dict(color="black")),
        meanline=dict(visible=False, color='black'),
    ))

for i, metric in enumerate(['FWCI', 'MNCS']):
    pairwise_fig.add_vline(x=metric, line=dict(color='black', dash='dash'))
    pairwise_fig.add_annotation(
        x=metric, xanchor="center", xref="x",
        y=1.04 if metric=="FWCI" else 1.06, yanchor="middle", yref="paper",
        text="*" if metric=="FWCI" else "n.s.",
        font=TITLE_FONT,
        font_size=30 if metric=="FWCI" else 20,
        showarrow=False,
    )
    pairwise_fig.add_shape(
        type="line",
        x0=i - 0.1, x1=i + 0.1,
        y0=1.025, y1=1.025,
        yref="paper",
        line=dict(color="black", width=1),
    )

pairwise_fig.update_layout(
    width=800, height=400,
    title=dict(
        text="<b>Citation Impact Distribution: Sharing vs. Non-Sharing</b>",
        font=TITLE_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top"
    ),
    violinmode='overlay',
    violingap=0,
    xaxis=dict(
        title=dict(text="<b>Metric</b>", font=AXIS_TITLE_FONT, standoff=5),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    ),
    yaxis=dict(
        title=dict(text="<b>Score</b>", font=AXIS_TITLE_FONT, standoff=5),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    ),
    legend=dict(
        title=dict(text="Data Sharing", font=AXIS_TITLE_FONT),
        font=LEGEND_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top",
        bgcolor='rgba(255,255,255,0.5)',
        bordercolor='black', borderwidth=1,
    ),
    margin=dict(t=75, b=50, l=50, r=25, pad=0),
    template="plotly_white"
)

pairwise_fig.show()

### (2) Compare Type of Data Sharing
#### (2A) Field-Weighted Citation Index (FWCI)

In [11]:
fwci_sharegroup_test = stats.kruskal(*[
    combined.loc[combined["data_sharing_class"] == sharing_class, "FieldWeightedCitationIndex"].dropna()
    for sharing_class in ["FIXATION", "TRIAL", "PPT"]
])
fwci_sharegroup_test

KruskalResult(statistic=np.float64(2.905931191024262), pvalue=np.float64(0.23387567797554926))

#### (2B) Mean Normalized Citation Score (MNCS)

In [12]:
mncs_sharegroup_test = stats.kruskal(*[
    combined.loc[combined["data_sharing_class"] == sharing_class, "MeanNormalizedCitationScore"].dropna()
    for sharing_class in ["FIXATION", "TRIAL", "PPT"]
])
mncs_sharegroup_test

KruskalResult(statistic=np.float64(2.505684267764051), pvalue=np.float64(0.28569166792766837))

#### Visualizing Share-Type Comparison

In [13]:
share_groups = {"FIXATION": "#66c2a5", "TRIAL": "#fc8d62", "PPT": "#8da0cb"}
metrics = {"FieldWeightedCitationIndex": "FWCI", "MeanNormalizedCitationScore": "MNCS",}
plot_data = (
    combined
    .loc[combined["data_sharing_class"].isin(share_groups.keys())]
    .melt(
        id_vars=['data_sharing_class'],
        value_vars=list(metrics.keys()),
        var_name='name',
        value_name='value'
    )
    .replace(metrics)
)

sharegroup_fig = make_subplots(
    rows=1, cols=len(metrics), column_titles=list(metrics.values()),
    shared_yaxes=True, horizontal_spacing=0.05,
)
annotation_height = plot_data["value"].max()
for c, (metric, metric_label) in enumerate(metrics.items(), start=1):
    for sharing_class, color in share_groups.items():
        subset = plot_data[
            (plot_data["data_sharing_class"] == sharing_class) &
            (plot_data["name"] == metric_label)
        ]
        sharegroup_fig.add_trace(
            row=1, col=c,
            trace=go.Violin(
                x=subset['data_sharing_class'], y=subset['value'],
                name=sharing_class, legendgroup=sharing_class, scalegroup=sharing_class,
                side='positive', points='all', pointpos=-0.25, jitter=0.2,
                fillcolor=color, line=dict(color=color, width=1.5),
                width=1.0, spanmode="hard",
                showlegend=c==1,
                box=dict(visible=False, width=0.5, line=dict(color="gray")),
                meanline=dict(visible=False, color='gray'),
            ))
    sharegroup_fig.update_annotations(
        selector={"text": metric_label},
        font=TITLE_FONT,
        x=-0.4, xanchor="left", xref="x" if c==1 else "x2",
        y=0.975, yanchor="bottom", yref="paper",
    )
    sharegroup_fig.add_annotation(
        row=1, col=c,
        x=1, xanchor="center", xref="x",
        y=annotation_height, yanchor="top", yref="y",
        text="n.s.", font=TITLE_FONT,
        showarrow=False,
    )
    # sharegroup_fig.add_shape(
    #     row=1, col=c,
    #     type="line",
    #     x0=0.75, x1=1.25,
    #     y0=annotation_height - 0.75, y1=annotation_height - 0.75,
    #     yref="y",
    #     line=dict(color="black", width=1),
    # )
    sharegroup_fig.update_xaxes(
        row=1, col=c,
        title=dict(text="<b>Type of Shared Data</b>", font=AXIS_TITLE_FONT, standoff=10),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    )
    sharegroup_fig.update_yaxes(
        row=1, col=c,
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    )

sharegroup_fig.update_layout(
    width=1000, height=400,
    title=dict(
        text="<b>Citation Impact by Type of Data Sharing</b>",
        font=TITLE_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top"
    ),
    yaxis=dict(
        title=dict(text="<b>Score</b>", font=AXIS_TITLE_FONT, standoff=5),
    ),
    legend=dict(
        visible=False,
        orientation="v",
        title=dict(text="Data Sharing", font=AXIS_TITLE_FONT),
        font=LEGEND_FONT,
        x=0.5, xanchor="center", y=1.0, yanchor="top",
        bgcolor='rgba(255,255,255,0.5)',
        bordercolor='black', borderwidth=1,
    ),
    margin=dict(t=75, b=50, l=50, r=25, pad=0),
    template="plotly_white"
)
sharegroup_fig.show()

## Citation Counts over Time
### (1) Regress each Sharing Group Separately

In [14]:
all_share_groups = {**share_groups, "NONE": "gray"}
citation_counts_fig = go.Figure()
for group, color in all_share_groups.items():
    subset = combined[combined["data_sharing_class"] == group]
    symbol = 'circle' if group != "NONE" else 'x'
    citation_counts_fig.add_trace(go.Scatter(
        name=group, legendgroup=group,
        x=subset["Pub2UpdateTime"] / pd.Timedelta(weeks=1), y=subset["TotalCitations"],
        mode="markers", marker=dict(color=color, size=8, symbol=symbol),
    ))
    # fit log-log trendline per group
    x_log = np.log(subset["Pub2UpdateTime"] / pd.Timedelta(weeks=1))
    y_log = np.log(subset['TotalCitations'] + 1)  # add 1 to avoid log(0)
    slope, intercept = np.polyfit(x_log, y_log, deg=1)
    x_logspace = np.linspace(x_log.min(), x_log.max(), 100)
    y_logpred = intercept + slope * x_logspace
    y_pred = np.exp(y_logpred) - 1  # invert log transform
    x_orig = np.exp(x_logspace)
    citation_counts_fig.add_trace(go.Scatter(
        name=f"Trendline ({group})",
        x=x_orig, y=y_pred, mode="lines", line=dict(color=color, width=2, dash='dot'),
        showlegend=False,
    ))
    print("Group: {0: <12}".format(group) + f"Slope: {slope:.3f},\tIntercept: {intercept:.2f}")

# add general log-log trendline
x_log = np.log(combined["Pub2UpdateTime"] / pd.Timedelta(weeks=1))
y_log = np.log(combined['TotalCitations'] + 1)
slope, intercept = np.polyfit(x_log, y_log, deg=1)
x_logspace = np.linspace(x_log.min(), x_log.max(), 100)
y_logpred = intercept + slope * x_logspace
y_pred = np.exp(y_logpred) - 1
x_orig = np.exp(x_logspace)
citation_counts_fig.add_trace(go.Scatter(
    name="Trendline (all data)",
    x=x_orig, y=y_pred, mode="lines", line=dict(color='black', width=2, dash='dash'),
))
print("{0: <19}".format("All Data:") + f"Slope: {slope:.3f},\tIntercept: {intercept:.2f}")

citation_counts_fig.update_layout(
    width=800, height=300,
    title=dict(
        text="Citation Counts over Time Since Publication", font=TITLE_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top",
    ),
    xaxis=dict(
        title=dict(text="Weeks Since Publication", font=AXIS_TITLE_FONT, standoff=10),
        tickfont=AXIS_TICK_FONT,
        # type="log",
        zeroline=False,
    ),
    yaxis=dict(
        title=dict(text="Total Citation Counts", font=AXIS_TITLE_FONT, standoff=5),
        tickfont=AXIS_TICK_FONT,
        # type="log",
        zeroline=True,
    ),
    legend=dict(
        orientation="h", bgcolor='rgba(0,0,0,0)',
        x=0.5, xanchor="center", y=0.96, yanchor="bottom",
    ),
    margin=dict(t=50, b=10, l=10, r=10, pad=0),
)
citation_counts_fig

Group: FIXATION    Slope: 1.498,	Intercept: -5.67
Group: TRIAL       Slope: 0.783,	Intercept: -1.90
Group: PPT         Slope: 1.360,	Intercept: -4.97
Group: NONE        Slope: 0.922,	Intercept: -2.66
All Data:          Slope: 1.018,	Intercept: -3.16


### (2) Quantify the Benefit of Data Sharing
We want to quantify the citation benefit of data sharing over time.<br>
To do so we first fit a log-log regression model to all data points, predicting citation counts from time since publication: $$\log(\text{Citations} +1) = \beta_0 + \beta_1 \log(\text{Weeks Since Publication}) + \epsilon$$(we add $1$ to the citation count to avoid $\log(0)$)<br><br>
Then, we compute the residuals (i.e., the difference between observed and predicted citation counts) for each article, and compare the residuals between sharing and non-sharing articles and across types of data shared.

In [15]:
# fit log-linear model to all data
x_log = np.log(combined["Pub2UpdateTime"] / pd.Timedelta(weeks=1))
y_log = np.log(combined['TotalCitations'] + 1)  # add 1 to avoid log(0)
slope, intercept = np.polyfit(x_log, y_log, deg=1)

# compute residuals
ylog_pred = intercept + slope * x_log
log_residuals = (y_log - ylog_pred).rename("citation_log_residuals")
combined_with_residuals = pd.concat([combined, log_residuals], axis=1)

# compare residuals between sharing and non-sharing articles
non_sharing_log_residuals = combined_with_residuals.loc[
    ~combined_with_residuals["is_sharing_data"], "citation_log_residuals"
]
sharing_log_residuals = combined_with_residuals.loc[
    combined_with_residuals["is_sharing_data"], "citation_log_residuals"
]
residuals_pair_test = stats.mannwhitneyu(
    non_sharing_log_residuals, sharing_log_residuals, alternative="less"
)

# calculate effect size (rank-biserial correlation / CLES)
n1, n2 = len(non_sharing_log_residuals), len(sharing_log_residuals)
U_nonshare = residuals_pair_test.statistic
U_share = n1 * n2 - U_nonshare
rank_biserial_corr = (U_share - U_nonshare) / (n1 * n2)
cles = U_share / (n1 * n2)

# calculate summary statistics
residuals_summary_stats = (
    combined_with_residuals
    .groupby("is_sharing_data")["citation_log_residuals"]
    .agg(["count", "median", "mean", "std"])
    .assign(geometric_mean=lambda df: np.exp(df["mean"]))
    .reset_index()
    .rename(columns={"is_sharing_data": "data_sharing"})
)

# show results
print(residuals_pair_test)
print(f"Effect Sizes:\t\tRBC: {rank_biserial_corr:.3f}\tCLES: {cles:.3f}")
residuals_summary_stats

MannwhitneyuResult(statistic=np.float64(5340.0), pvalue=np.float64(0.07337071308811977))
Effect Sizes:		RBC: 0.116	CLES: 0.558


,data_sharing,count,median,mean,std,geometric_mean
0,False,153,-0.009186,-0.057329,0.840055,0.944284
1,True,79,0.159878,0.111029,0.855813,1.117427


## Citation Dynamics
We further explor how citations differ **over time** between sharing and non-sharing articles; and whether different types of data sharing are associated with different citation dynamics.<br>
To do so, we calculate the number of citations in each calendar year post-publication for each article (ignoring "year 0" due to partial year effects). We then average these yearly citation counts across articles within each sharing category, and examine the differences.

In [116]:
def citations_per_year(
        articles: pd.DataFrame, grouping_col: str, cumulative: bool = True
) -> pd.DataFrame:
    """
    Extract citation counts per calendar-year post publication from OpenAlex metadata.
    Citations are stored in columns "year_X" where X is the number of years since publication, e.g. "year_0"
    for citations from the same year as article's publication.
    We ignore the article's publication year (year 0) and the current year (2026) to avoid
    partial-year effects.

    :param articles: pd.Datarme containing the grouping column and additional columns:
        - 'Citations20XX' columns with citation counts per year
        - 'PublicationYear' column with the publication year of each article
    :param grouping_col: str, name of the column to group by (e.g., "is_sharing_data" or "data_sharing_class")
    :param cumulative: bool, whether to return cumulative citation counts (default: True)
    :return: pd.DataFrame with columns "year_X" and the same index as the input articles DataFrame
    """
    cit_cols = [c for c in articles.columns if c.startswith("Citations20") and not c.endswith("2026")]
    articles_long = articles.reset_index().melt(
        id_vars=['index', grouping_col, 'PublicationYear'],
        value_vars=cit_cols,
        var_name='CitationYear_Str',
        value_name='Count'
    )
    articles_long['CitationYear'] = articles_long['CitationYear_Str'].str.extract(r'(\d{4})').astype(int)
    articles_long['RelYear'] = articles_long['CitationYear'] - articles_long['PublicationYear']
    articles_long = articles_long.loc[articles_long['RelYear'] > 0]    # drop citations from before publication year
    articles_long['Count'] = articles_long['Count'].fillna(0)
    cit_dynamics = articles_long.pivot_table(
        index=['index', grouping_col],
        columns='RelYear',
        values='Count',
        aggfunc='sum'
    )
    cit_dynamics.columns = [int(c) for c in cit_dynamics.columns]
    if cumulative:
        cit_dynamics = cit_dynamics.cumsum(axis=1)
    return cit_dynamics

### (1) Sharing vs. Non-Sharing Articles

In [117]:
GROUPING_COL = "is_sharing_data"

In [118]:
cit_dynamics_sharing = citations_per_year(
    combined, grouping_col=GROUPING_COL, cumulative=True
)

year_stats = dict()
for year in cit_dynamics_sharing.columns:
    non_sharing_vals = cit_dynamics_sharing.xs(False, level=GROUPING_COL)[year].dropna()
    sharing_vals = cit_dynamics_sharing.xs(True, level=GROUPING_COL)[year].dropna()
    n1, n2 = len(non_sharing_vals), len(sharing_vals)
    year_pair_test = stats.mannwhitneyu(non_sharing_vals, sharing_vals, alternative="less")
    U_nonshare = year_pair_test.statistic
    U_share = n1 * n2 - U_nonshare
    rank_biserial_corr = (U_share - U_nonshare) / (n1 * n2)
    cles = U_share / (n1 * n2)
    year_stats[year] = {
        "non_sharing_count": n1,
        "non_sharing_mean": non_sharing_vals.mean(),
        "non_sharing_sem": non_sharing_vals.sem(),
        "sharing_count": n2,
        "sharing_mean": sharing_vals.mean(),
        "sharing_sem": sharing_vals.sem(),
        "U_nonshare": U_nonshare,
        "p_raw": year_pair_test.pvalue,
        "r_rb": rank_biserial_corr,
        "cles": cles,
    }

year_stats = pd.DataFrame.from_dict(year_stats, orient='index').reset_index().rename(columns={"index": "year"})
reject, p_corrected, _, _ = multipletests(year_stats['p_raw'], alpha=0.05, method='fdr_bh')
year_stats['p_corrected'] = p_corrected
year_stats['is_significant'] = reject
column_order = [
    "year",
    "is_significant", "p_corrected", "U_nonshare", "r_rb", "cles",
    "non_sharing_count", "non_sharing_mean", "non_sharing_sem",
    "sharing_count", "sharing_mean", "sharing_sem",
    "p_raw"
]
year_stats = year_stats[column_order]

year_stats

,year,is_significant,p_corrected,U_nonshare,r_rb,cles,non_sharing_count,non_sharing_mean,non_sharing_sem,sharing_count,sharing_mean,sharing_sem,p_raw
0,1,True,0.036943,4782.0,0.208737,0.604368,153,2.444444,0.210019,79,3.670886,0.527320,0.004105
1,2,True,0.039255,5000.0,0.172665,0.586332,153,5.888889,0.466412,79,7.886076,0.802258,0.015385
2,3,True,0.039255,5022.5,0.168942,0.584471,153,9.202614,0.682057,79,12.101266,1.130785,0.017447
3,4,True,0.039255,3988.5,0.186020,0.593010,140,12.542857,0.977042,70,16.357143,1.571490,0.014027
4,5,False,0.095934,2257.5,0.149718,0.574859,118,15.177966,1.231888,45,19.600000,2.495531,0.070113
5,6,False,0.095934,1205.5,0.172047,0.586023,91,18.483516,1.650128,32,24.718750,3.774980,0.074615
6,7,False,0.085435,453.5,0.254112,0.627056,64,22.218750,2.303430,19,32.000000,6.010225,0.047464
7,8,False,0.226281,185.5,0.164414,0.582207,37,25.486486,3.347408,12,31.250000,6.251212,0.201139
8,9,False,0.355556,6.0,0.250000,0.625000,8,35.500000,12.561278,2,35.500000,12.500000,0.355556


In [119]:
LINE_ANNOTATION_X = 3

cit_dynamics_sharing_fig = go.Figure()
for is_share in cit_dynamics_sharing.index.get_level_values(GROUPING_COL).unique().sort_values():
    subset = cit_dynamics_sharing.xs(is_share, level=GROUPING_COL)
    count_per_year = subset.count(axis=0)
    mean_per_year = subset.mean(axis=0)
    sem_per_year = subset.sem(axis=0)
    mean_citations_per_year = subset.mean(axis=0)
    label = "Sharing" if is_share else "Not Sharing"
    color = "gray" if is_share else "lightgray"
    cit_dynamics_sharing_fig.add_trace(go.Scatter(
        name=label, legendgroup=label,
        x=mean_per_year.index,
        y=mean_per_year.values,
        error_y=dict(type='data', array=sem_per_year.values, width=5, visible=True,),
        marker=dict(
            color=color,
            # size=np.sqrt(count_per_year.values),
        ),
        mode='lines+markers',
    ))
    # annotate the lines
    if is_share:
        ann_y = mean_per_year[LINE_ANNOTATION_X] + 1.5 * sem_per_year[LINE_ANNOTATION_X]
        xanchor = "right"
        yanchor = "bottom"
    else:
        ann_y = mean_per_year[LINE_ANNOTATION_X] - 1.5 * sem_per_year[LINE_ANNOTATION_X]
        xanchor = "left"
        yanchor = "top"
    cit_dynamics_sharing_fig.add_annotation(
        x=LINE_ANNOTATION_X, xanchor=xanchor, xref="x",
        y=ann_y, yanchor=yanchor, yref="y",
        text=label, font={**AXIS_TITLE_FONT, "color": color},
        showarrow=False,
    )

# add significance asterisks
SIGNIFICANCE_ANNOTATION_Y = np.floor(
    year_stats.loc[0, "non_sharing_mean"] - year_stats.loc[0, "non_sharing_sem"]
)
SIGNIFICANCE_ANNOTATION_Y = 40
for _, row in year_stats.iterrows():
    if row["p_corrected"] >= 0.1:
        continue
    fontsize = 22
    yanchor = "middle"
    if row["p_corrected"] < 0.001:
        ann_text = "<b>***</b>"
    elif row["p_corrected"] < 0.01:
        ann_text = "<b>**</b>"
    elif row["p_corrected"] < 0.05:
        ann_text = "<b>*</b>"
    else:
        ann_text = "\u2020"     # dagger for p < 0.
        fontsize = 16
        yanchor = "bottom"
    cit_dynamics_sharing_fig.add_annotation(
        x=row["year"], xanchor="center", xref="x",
        y=SIGNIFICANCE_ANNOTATION_Y, yanchor=yanchor, yref="y",
        text=ann_text, font={**TITLE_FONT, "size": fontsize},
        showarrow=False,
    )

cit_dynamics_sharing_fig.update_layout(
    width=800, height=400,
    title=dict(
        text="Citation Dynamics: Sharing vs. Non-Sharing Articles",
        font=TITLE_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top"
    ),
    xaxis=dict(
        title=dict(text="Years Since Publication", font=AXIS_TITLE_FONT, standoff=10),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    ),
    yaxis=dict(
        title=dict(text="Mean Cumulative Citation per Year", font=AXIS_TITLE_FONT, standoff=5),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    ),
    legend=dict(
        title=dict(text="Data Sharing", font=AXIS_TITLE_FONT),
        font=LEGEND_FONT,
        x=1.0, xanchor="right", y=0.4, yanchor="top",
        bgcolor='rgba(255,255,255,0.5)',
        bordercolor='black', borderwidth=1,
        visible=False,
    ),
    margin=dict(t=75, b=50, l=50, r=25, pad=0),
    template="plotly_white"
)
cit_dynamics_sharing_fig.show()

### (2) Different Types of Data Sharing

In [ ]:
GROUPING_COL = "data_sharing_class"

In [67]:
cit_dynamics_sharing_class = citations_per_year(
    combined.loc[combined["is_sharing_data"]], grouping_col=GROUPING_COL
)

year_stats = dict()
for year in cit_dynamics_sharing_class.columns:
    year_stats[year] = dict()
    class_values = []
    for share_class in cit_dynamics_sharing_class.index.get_level_values(GROUPING_COL).unique():
        subset = (cit_dynamics_sharing_class.xs(share_class, level=GROUPING_COL)[year]).dropna()
        if subset.empty:
            continue
        year_stats[year][f"{share_class}_count"] = subset.count()
        year_stats[year][f"{share_class}_mean"] = subset.mean()
        year_stats[year][f"{share_class}_sem"] = subset.sem()
        class_values.append(subset)
    sharegroup_test = stats.kruskal(*class_values)
    year_stats[year]["W"] = sharegroup_test.statistic
    year_stats[year]["p_raw"] = sharegroup_test.pvalue

year_stats = pd.DataFrame.from_dict(year_stats, orient='index')
reject, p_corrected, _, _ = multipletests(year_stats['p_raw'], alpha=0.05, method='fdr_bh')
year_stats['p_corrected'] = p_corrected
year_stats['is_significant'] = reject
column_order = [
    "is_significant", "p_corrected", "W", "p_raw",
    *[f"{share_class}_{stat}" for share_class in share_groups.keys() for stat in ["count", "mean", "sem"]]
]
year_stats = year_stats[column_order]
year_stats

,is_significant,p_corrected,W,p_raw,FIXATION_count,FIXATION_mean,FIXATION_sem,TRIAL_count,TRIAL_mean,TRIAL_sem,PPT_count,PPT_mean,PPT_sem
1,False,0.481202,3.589866,0.166139,31,4.741935,1.198537,25.0,2.520000,0.513550,23,3.478261,0.543992
2,False,0.481202,2.124948,0.345600,31,9.129032,1.506789,25.0,6.760000,1.381159,23,7.434783,1.111965
3,False,0.481202,2.142026,0.342661,31,13.322581,1.877648,25.0,10.360000,2.058219,23,12.347826,1.955643
4,False,0.481202,1.867384,0.393100,28,17.892857,2.473852,23.0,14.521739,2.965460,19,16.315789,2.813972
5,False,0.481202,1.571074,0.455875,21,21.666667,3.573936,11.0,15.272727,4.794108,13,19.923077,5.152124
6,False,0.481202,2.288096,0.318527,15,28.533333,5.439596,6.0,14.500000,4.667262,11,25.090909,7.641340
7,False,0.481202,1.462937,0.481202,8,33.250000,7.694502,4.0,19.750000,6.289873,7,37.571429,13.569674
8,False,0.481202,1.488722,0.475038,7,39.428571,9.195903,2.0,23.500000,14.500000,3,17.333333,1.666667
9,False,0.481202,1.000000,0.317311,1,48.000000,NaN,NaN,NaN,NaN,1,23.000000,NaN


In [66]:
cit_dynamics_sharing_class_fig = go.Figure()
for share_class in cit_dynamics_sharing_class.index.get_level_values(GROUPING_COL).unique():
    subset = cit_dynamics_sharing_class.xs(share_class, level=GROUPING_COL)
    count_per_year = subset.count(axis=0)
    mean_per_year = subset.mean(axis=0)
    sem_per_year = subset.sem(axis=0)
    mean_citations_per_year = subset.mean(axis=0)
    cit_dynamics_sharing_class_fig.add_trace(go.Scatter(
        name=share_class, legendgroup=share_class,
        x=mean_per_year.index,
        y=mean_per_year.values,
        error_y=dict(type='data', array=sem_per_year.values, width=5, visible=True,),
        marker=dict(
            color=share_groups[share_class],
            size=count_per_year.values
        ),
        mode='lines+markers',
    ))

cit_dynamics_sharing_class_fig.update_layout(
    width=800, height=400,
    title=dict(
        text="Citation Dynamics: Different Types of Data Sharing",
        font=TITLE_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top"
    ),
    xaxis=dict(
        title=dict(text="Years Since Publication", font=AXIS_TITLE_FONT, standoff=10),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    ),
    yaxis=dict(
        title=dict(text="Mean Cumulative Citation per Year", font=AXIS_TITLE_FONT, standoff=5),
        tickfont=AXIS_TICK_FONT,
        zeroline=False,
    ),
    legend=dict(
        title=dict(text="Data Sharing", font=AXIS_TITLE_FONT),
        font=LEGEND_FONT,
        x=0.5, xanchor="center", y=0.95, yanchor="top",
        bgcolor='rgba(255,255,255,0.5)',
        bordercolor='black', borderwidth=1,
    ),
    margin=dict(t=75, b=50, l=50, r=25, pad=0),
    template="plotly_white"
)
cit_dynamics_sharing_class_fig.show()